In [74]:
import pandas as pd
import numpy as np
# import stanfordnlp
import time
import scispacy
from tqdm import tqdm
import itertools
import pickle
import sys
import re, nltk
import os
from heuristic_tokenize import sent_tokenize_rules
import warnings
warnings.filterwarnings("ignore")

In [75]:
#setting sentence boundaries
def sbd_component(doc):
    for i, token in enumerate(doc[:-2]):
        # define sentence start if period + titlecase token
        if token.text == '.' and doc[i+1].is_title:
            doc[i+1].sent_start = True
        if token.text == '-' and doc[i+1].text != '-':
            doc[i+1].sent_start = True
    return doc

#convert de-identification text into one token
def fix_deid_tokens(text, processed_text):
    deid_regex  = r"\[\*\*.{0,15}.*?\*\*\]" 
    if text:
        #print("!!!!!")
        indexes = [m.span() for m in re.finditer(deid_regex,text,flags=re.IGNORECASE)]
    else:
        #print("??????")
        indexes = []
    #print(indexes)
    for start,end in indexes:
        a = processed_text.merge(start_idx=start,end_idx=end)
        #print(a)
    #print(text)
    return processed_text
    

def process_section(section, note, processed_sections):
    # perform spacy processing on section
    #print("--------- before ---------")
    #print(section['sections'])
    processed_section = nlp(section['sections'])
    #print("--------- AFTER ----------")
    #print(processed_section)
    processed_section = fix_deid_tokens(section['sections'], processed_section)
    processed_sections.append(processed_section)

def process_note_helper(note):
    # split note into sections
    note_sections = sent_tokenize_rules(note)
    processed_sections = []
    section_frame = pd.DataFrame({'sections':note_sections})
    
    section_frame.apply(process_section, args=(note,processed_sections,), axis=1)
    return(processed_sections)

def process_text(sent, note):
    sent_text = sent['sents'].text
    sent_text = sent_text.replace("_","")
    sent_text = sent_text.replace("#","")
    sent_text = re.sub(r"(1[0-2]|[1-9]):[0-5][0-9] *(a|p|A|P)(m|M) *", "",sent_text)
    if len(sent_text) > 0 and sent_text.strip() != '\n':
        if '\n' in sent_text:
            sent_text = sent_text.replace('\n', ' ')
#         print('sent_text:', sent_text)
        if sent_text[-1] ==':':
            note['HEADERS'] += sent_text[:-1] + ','
#         else:
        note['TEXT'] += sent_text + '\n'
    
    
    

def get_sentences(processed_section, note):
    # get sentences from spacy processing
    sent_frame = pd.DataFrame({'sents': list(processed_section['sections'].sents)})
    sent_frame.apply(process_text, args=(note,), axis=1)


def process_note(note):
    try:
        note_text = note['TEXT'] #unicode(note['text'])
        note['TEXT'] = ''
        processed_sections = process_note_helper(note_text)
        ps = {'sections': processed_sections}
        ps = pd.DataFrame(ps)
        ps.apply(get_sentences, args=(note,), axis=1)
        return note 
    except Exception as e:
        print('!!!!! error', e)
        pass 

# **en_core_sci_md:-**
A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.	

In [76]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [77]:
import spacy

In [78]:
# !python -m spacy download en_core_web_sm

In [79]:
nlp=spacy.load('en_core_web_sm')

In [80]:
# import en_core_sci_md
# nlp = en_core_sci_md.load()
nlp = spacy.load('en_core_sci_md', disable=['tagger','ner'])
nlp.add_pipe(sbd_component, before='parser')

OSError: [E050] Can't find model 'en_core_sci_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [10]:
SAMPLE_20K = pd.read_csv(r'D:\FINALYEARPROJECTREC\artifacts\SAMPLE_20K.csv')

In [11]:
SAMPLE_20K['TEXT'] = SAMPLE_20K['TEXT'].replace(to_replace=r"\[.*?\]", value="", regex=True)

In [16]:
SAMPLE_20K

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,HEADERS
0,146115.0,Admission Date: Discharge Date:...,Discharge summary Echo ECG ECG ECG Nursing Nur...,"4920,51881,4150,7994,2762,49121,42830,4280,238...",NaN,
1,121889.0,Admission Date: Discharge Date:...,Discharge summary ECG ECG ECG ECG Radiology Ra...,"5849,5609,2767,4280,42731,20300,5789,42732,414...",NaN,
2,170374.0,Admission Date: Discharge Date:...,Discharge summary ECG Radiology Radiology Nurs...,"430,5990,4019,43490,34690",NaN,
3,191619.0,Admission Date: Discharge Date:...,Discharge summary Echo Echo ECG ECG ECG ECG Ra...,"41071,4280,496,5849,5781,78551,41401,V1005,V10...",NaN,
4,128946.0,Admission Date: Discharge Date:...,Discharge summary Echo ECG ECG Radiology Radio...,"43491,78003,7843,725,4019,41401,412,2724,4264",NaN,
...,...,...,...,...,...,...
19995,178275.0,Admission Date: Discharge Date: Date...,Discharge summary Discharge summary Discharge ...,"5849,5728,5712,5990,0417,2910,03819,42731,5070",NaN,
19996,104018.0,Family Information Next of : (mother) ...,Social Work Nursing Nursing Nursing Nursing Nu...,"88000,9038,E966",NaN,
19997,195908.0,Admission Date: Discharge Date: Date o...,Discharge summary Echo ECG ECG ECG Radiology R...,"5738,4590,7895,79579,2875,7100,58281,585,28521",NaN,
19998,173420.0,Admission Date: Discharge Date:...,Discharge summary ECG,"99859,73018,99883,7907,04111,0417,41072,25000,...",NaN,


In [14]:
# SAMPLE_20K = SAMPLE_20K.drop(columns=['Unnamed: 0'])

In [15]:
SAMPLE_20K['HEADERS'] = ""

In [17]:
print('Number of notes: %d' %len(SAMPLE_20K.index))
SAMPLE_20K['ind'] = list(range(len(SAMPLE_20K.index)))

Number of notes: 20000


In [18]:
tqdm.pandas()

In [19]:
SAMPLE_20K = SAMPLE_20K.progress_apply(process_note, axis=1)

  0%|          | 28/20000 [00:43<5:46:02,  1.04s/it] 

!!!!! error list index out of range


  0%|          | 34/20000 [00:48<4:04:32,  1.36it/s]

!!!!! error list index out of range


  0%|          | 36/20000 [00:48<2:49:13,  1.97it/s]

...

100%|█████████▉| 19984/20000 [6:14:55<00:12,  1.26it/s]

!!!!! error list index out of range


100%|█████████▉| 19999/20000 [6:15:10<00:01,  1.44s/it]

!!!!! error list index out of range
!!!!! error list index out of range


100%|██████████| 20000/20000 [6:15:12<00:00,  1.13s/it]


In [ ]:
hgc

In [22]:
keep = SAMPLE_20K.dropna()

In [23]:
keep.head()

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,HEADERS,ind
585,125157.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG Physician Nursing Nursi...,"1510,5185,4019,2724","4399.0,4223.0,3491.0,966.0","Admission Date,Discharge Date,",585.0
4408,145024.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Radiology Radiology Radiolog...,"53642,5990,3441,51909,3572,E8798,43853,43810,4...","9723.0,9702.0,3893.0","Admission Date,Discharge Date,",4408.0
4662,130870.0,Admission Date: Discharge Date: Date of Bir...,Discharge summary Discharge summary Discharge ...,"1510,9971,9974,51881,51189,42731,2875,47831,51...","4399.0,4252.0,966.0,9604.0,9672.0,3404.0,3324....","PSH: spinal fusion, s/p orchidectomy for torsi...",4662.0
5067,160442.0,Admission Date:\nDischarge Date:\nFINAL REPORT...,Discharge summary Radiology,"86404,8504,9584,4275,2851,E8232,86389,73689","5061.0,3712.0,3404.0,9960.0,9604.0,9671.0,9904.0","Admission Date,Discharge Date,",5067.0
6234,167181.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG ECG ECG Physician N...,"1541,5849,1977,1962,1976,5990,78959,5772,2800,...",4613.0,"Admission Date,Discharge Date,",6234.0
6486,102203.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Nursing Physician Physician...,"5722,5849,78959,261,5724,2763,2762,5712,53789,...","966.0,5491.0","Admission Date,Discharge Date,WET READ,",6486.0
6521,125449.0,PATIENT/TEST INFORMATION:\n-----\n< 4.3 27...,Echo ECG ECG Nursing Nursing Nursing Nursing N...,"0389,5070,51881,5849,99592,4019,25000,2449,285...","4432.0,9604.0,9671.0,3893.0,966.0","PATIENT/TEST INFORMATION, PPX,ICU Care Nutr...",6521.0
7382,125013.0,Admission Date:\nDischarge Date:\nPROTEUS SPEC...,Discharge summary Echo ECG Radiology Radiology...,"311,42731,V5861,V1254,V440,53081,03849,78552,5...",3893.0,"Admission Date,Discharge Date,gabapentin 300 m...",7382.0
7746,114648.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG Physician Nursing N...,"5990,5849,8600,75321,2760,2930,2851,42731,5859...","3893.0,3404.0,9907.0,9904.0","Admission Date,Discharge Date,NOTE ADDED AT AT...",7746.0
8654,178513.0,Admission Date:\nDischarge Date:\n-----\nProte...,Discharge summary Echo ECG ECG ECG Physician ...,"41071,43401,34290,1570,1977,1970,99812,4531,79...","3974.0,8841.0,9910.0,41.0","Admission Date,Discharge Date,Tmax: 37.8 C (10...",8654.0


In [24]:
discard_lst = {'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
               'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
               ' MD Phone',' M.D. Phone', ' MD',' PHD',
               ' X', ' IV', ' VI', 'III', 'II', 'VIII',
               'JOB#','JOB#: cc', '# Code',
               'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
              }
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

In [25]:
# get top 10 sentences and get 25 vocabularies from those sentences
all_tokens = []
num_vocabs = 25
num_sentences = 10
discard_tokens = ('Sex: F', 'Sex: M', 'MEDICINE', 'Neurosurgery', 'Service: CT', 'Allergies')
for index, row in keep.iterrows():
    cur = [ ["<PAD>"] * num_vocabs for i in range(num_sentences) ]
    sents = row["TEXT"].splitlines()
    headers = ' '.join(row['HEADERS']).split()
        
    """
    for j in range(len(headers)):
        if j < num_vocabs:
            cur[0][j] = headers[j]       
    """          
    
    sentence_idx = 0
    for i in range(0, len(sents)):
        if sentence_idx >= num_sentences:
            break
        
        if sents[i].startswith(discard_tokens):
            continue
        
        vocabs = sents[i].split()
        
        for j in range(len(vocabs)):  
            if(j >= num_vocabs):
                break
            
            #print(str(sentence_idx)+" "+str(j)+" "+vocabs[j])
            cur[sentence_idx][j] = vocabs[j]
        
        sentence_idx += 1
    
    all_tokens.append(list(itertools.chain.from_iterable(cur)))
keep['padded_tokens'] = all_tokens

In [26]:
keep

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,HEADERS,ind,padded_tokens
585,125157.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG Physician Nursing Nursi...,"1510,5185,4019,2724","4399.0,4223.0,3491.0,966.0",[],585.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
4408,145024.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Radiology Radiology Radiolog...,"53642,5990,3441,51909,3572,E8798,43853,43810,4...","9723.0,9702.0,3893.0",[],4408.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
4662,130870.0,Admission Date: Discharge Date: Date of Bir...,Discharge summary Discharge summary Discharge ...,"1510,9971,9974,51881,51189,42731,2875,47831,51...","4399.0,4252.0,966.0,9604.0,9672.0,3404.0,3324....","[WET READ, Now on CPAP Gastrointestinal ...",4662.0,"[Admission, Date:, Discharge, Date:, Date, of,..."
5067,160442.0,Admission Date:\nDischarge Date:\nFINAL REPORT...,Discharge summary Radiology,"86404,8504,9584,4275,2851,E8232,86389,73689","5061.0,3712.0,3404.0,9960.0,9604.0,9671.0,9904.0",[],5067.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
6234,167181.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG ECG ECG Physician N...,"1541,5849,1977,1962,1976,5990,78959,5772,2800,...",4613.0,[],6234.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
6486,102203.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Nursing Physician Physician...,"5722,5849,78959,261,5724,2763,2762,5712,53789,...","966.0,5491.0",[WET READ],6486.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
6521,125449.0,PATIENT/TEST INFORMATION:\n-----\n< 4.3 27...,Echo ECG ECG Nursing Nursing Nursing Nursing N...,"0389,5070,51881,5849,99592,4019,25000,2449,285...","4432.0,9604.0,9671.0,3893.0,966.0","[WET READ, / Airway problem...",6521.0,"[PATIENT/TEST, INFORMATION:, <PAD>, <PAD>, <PA..."
7382,125013.0,Admission Date:\nDischarge Date:\nPROTEUS SPEC...,Discharge summary Echo ECG Radiology Radiology...,"311,42731,V5861,V1254,V440,53081,03849,78552,5...",3893.0,"[bisacodyl 5 mg Tablet, Date/Time: 10:00 PATI...",7382.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
7746,114648.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG Physician Nursing N...,"5990,5849,8600,75321,2760,2930,2851,42731,5859...","3893.0,3404.0,9907.0,9904.0","[NOTE ADDED AT ATTENDING REVIEW, IMPRESSION]",7746.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."
8654,178513.0,Admission Date:\nDischarge Date:\n-----\nProte...,Discharge summary Echo ECG ECG ECG Physician ...,"41071,43401,34290,1570,1977,1970,99812,4531,79...","3974.0,8841.0,9910.0,41.0",[Tmax: 37.3 C (99.2 T current: 36.2 C (97.1...,8654.0,"[Admission, Date:, <PAD>, <PAD>, <PAD>, <PAD>,..."


In [27]:
keep.to_pickle(r"D:\FINALYEARPROJECTREC\artifacts\abhi.pkl")
keep.to_csv(r'D:\FINALYEARPROJECTREC\artifacts\abhi.csv',index=True)

In [28]:
keep = pd.read_csv(r"D:\FINALYEARPROJECTREC\artifacts\abhi.csv")

In [29]:
keep = keep.drop(columns=['Unnamed: 0'])

In [30]:
keep2 = SAMPLE_20K.dropna()

In [31]:
def clean(text):
    output=re.sub(r'\n',' ',text)
    output = re.sub(r" +", " ",output)
    output = output.strip()
    return output

In [32]:
keep['CLEANED TEXT']=keep['TEXT'].apply(clean)


In [33]:
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 
    
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [73]:
discard_list={'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
            'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
            ' MD Phone',' M.D. Phone', ' MD',' PHD',
            ' X', ' IV', ' VI', 'III', 'II', 'VIII',
            'JOB#','JOB#: cc', '# Code',
            'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
            }
# keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

KeyError: 'HEADERS'

In [72]:
keep

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,ind,padded_tokens,CLEANED TEXT,CLEAN_WORDS,STEM_WORDS,CODED_TEXT
0,125157.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG Physician Nursing Nursi...,"1510,5185,4019,2724","4399.0,4223.0,3491.0,966.0",585.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 255, 24, 5..."
1,145024.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Radiology Radiology Radiolog...,"53642,5990,3441,51909,3572,E8798,43853,43810,4...","9723.0,9702.0,3893.0",4408.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 255, 139, ..."
2,130870.0,Admission Date: Discharge Date: Date of Bir...,Discharge summary Discharge summary Discharge ...,"1510,9971,9974,51881,51189,42731,2875,47831,51...","4399.0,4252.0,966.0,9604.0,9672.0,3404.0,3324....",4662.0,"['Admission', 'Date:', 'Discharge', 'Date:', '...",Admission Date: Discharge Date: Date of Birth:...,"[admission, date, discharge, date, date, birth...","[admiss, date, discharg, date, date, birth, se...","[66, 230, 359, 230, 230, 3661, 3662, 529, 565,..."
3,160442.0,Admission Date:\nDischarge Date:\nFINAL REPORT...,Discharge summary Radiology,"86404,8504,9584,4275,2851,E8232,86389,73689","5061.0,3712.0,3404.0,9960.0,9604.0,9671.0,9904.0",5067.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: FINAL REPORT I...,"[admission, date, discharge, date, final, repo...","[admiss, date, discharg, date, final, report, ...","[66, 230, 359, 230, 109, 73, 281, 1466, 175, 2..."
4,167181.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG ECG ECG Physician N...,"1541,5849,1977,1962,1976,5990,78959,5772,2800,...",4613.0,6234.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 367, 848, ..."
5,102203.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Nursing Physician Physician...,"5722,5849,78959,261,5724,2763,2762,5712,53789,...","966.0,5491.0",6486.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 255, 2596,..."
6,125449.0,PATIENT/TEST INFORMATION:\n-----\n< 4.3 27...,Echo ECG ECG Nursing Nursing Nursing Nursing N...,"0389,5070,51881,5849,99592,4019,25000,2449,285...","4432.0,9604.0,9671.0,3893.0,966.0",6521.0,"['PATIENT/TEST', 'INFORMATION:', '<PAD>', '<PA...",PATIENT/TEST INFORMATION: ----- < 4.3 27 1.1 e...,"[patienttest, information, estgfr, ca, mg, p, ...","[patienttest, inform, estgfr, ca, mg, p, alt, ...","[3040, 1058, 5850, 186, 11, 473, 388, 179, 304..."
7,125013.0,Admission Date:\nDischarge Date:\nPROTEUS SPEC...,Discharge summary Echo ECG Radiology Radiology...,"311,42731,V5861,V1254,V440,53081,03849,78552,5...",3893.0,7382.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: PROTEUS SPECIE...,"[admission, date, discharge, date, proteus, sp...","[admiss, date, discharg, date, proteu, speci, ...","[66, 230, 359, 230, 1557, 4778, 1476, 2097, 13..."
8,114648.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG Physician Nursing N...,"5990,5849,8600,75321,2760,2930,2851,42731,5859...","3893.0,3404.0,9907.0,9904.0",7746.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, d

In [35]:
keep['CLEAN_WORDS']=keep['TEXT'].apply(preprocess)

In [36]:
from nltk.stem import *
stemmer=PorterStemmer()

In [37]:
keep['STEM_WORDS']=keep['CLEAN_WORDS'].apply(lambda x: [stemmer.stem(w) for w in x])

In [38]:
corpus={}
for i,s in enumerate(keep['STEM_WORDS']):
    for w in s:
        corpus[w]=corpus.get(w,1)+1
corpus={k: v for k,v in sorted(corpus.items(),key=lambda item: item[1],reverse=True)}


In [39]:
len(corpus)

8069

In [40]:
corpus_slice=dict(itertools.islice(corpus.items(),10000))

In [41]:
len(corpus_slice)

8069

In [42]:
# build dictionary
word2idx = {'<PAD>': 0, '<UNK>':1}
idx2word = {0: '<PAD>', 1:'<UNK>'}
for c in corpus_slice:
    word2idx[c] = len(word2idx)
    idx2word[len(idx2word)] = c

In [43]:
length = 256
def note2idx_cap(org_lst):
    coded_lst = []
    for w in org_lst:
        if len(coded_lst) < length and w in word2idx:
            coded_lst.append(word2idx[w])
        else:
            coded_lst.append(1)
    coded_lst += [0]*(length-len(coded_lst))
    return coded_lst

In [47]:
keep['CODED_TEXT']=keep['STEM_WORDS'].apply(note2idx_cap)

In [48]:
keep=keep.drop(columns=['HEADERS'])

In [49]:
keep.head()

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,ind,padded_tokens,CLEANED TEXT,CLEAN_WORDS,STEM_WORDS,CODED_TEXT
0,125157.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG Physician Nursing Nursi...,"1510,5185,4019,2724","4399.0,4223.0,3491.0,966.0",585.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 255, 24, 5..."
1,145024.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary Radiology Radiology Radiolog...,"53642,5990,3441,51909,3572,E8798,43853,43810,4...","9723.0,9702.0,3893.0",4408.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 255, 139, ..."
2,130870.0,Admission Date: Discharge Date: Date of Bir...,Discharge summary Discharge summary Discharge ...,"1510,9971,9974,51881,51189,42731,2875,47831,51...","4399.0,4252.0,966.0,9604.0,9672.0,3404.0,3324....",4662.0,"['Admission', 'Date:', 'Discharge', 'Date:', '...",Admission Date: Discharge Date: Date of Birth:...,"[admission, date, discharge, date, date, birth...","[admiss, date, discharg, date, date, birth, se...","[66, 230, 359, 230, 230, 3661, 3662, 529, 565,..."
3,160442.0,Admission Date:\nDischarge Date:\nFINAL REPORT...,Discharge summary Radiology,"86404,8504,9584,4275,2851,E8232,86389,73689","5061.0,3712.0,3404.0,9960.0,9604.0,9671.0,9904.0",5067.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: FINAL REPORT I...,"[admission, date, discharge, date, final, repo...","[admiss, date, discharg, date, final, report, ...","[66, 230, 359, 230, 109, 73, 281, 1466, 175, 2..."
4,167181.0,Admission Date:\nDischarge Date:\nMEDICAL COND...,Discharge summary ECG ECG ECG ECG Physician N...,"1541,5849,1977,1962,1976,5990,78959,5772,2800,...",4613.0,6234.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: MEDICAL CONDIT...,"[admission, date, discharge, date, medical, co...","[admiss, date, discharg, date, medic, condit, ...","[66, 230, 359, 230, 9, 151, 96, 81, 367, 848, ..."


In [50]:
keep.to_csv(r'D:\FINALYEARPROJECTREC\artifacts\May6.csv',index=True)

In [71]:
keep.columns

Index(['HADM_ID', 'TEXT', 'CATEGORY', 'DIAG_CODES', 'PROC_CODES', 'ind',
       'padded_tokens', 'CLEANED TEXT', 'CLEAN_WORDS', 'STEM_WORDS',
       'CODED_TEXT'],
      dtype='object')

In [ ]:
HADM_ID', 'TEXT', 'CATEGORY', 'DIAG_CODES', 'PROC_CODES', 'ind',

In [53]:
with open('word2idx.pickle','wb') as f:
    pickle.dump(word2idx,f,pickle.HIGHEST_PROTOCOL)
with open('idx2word.pickle','wb') as f:
    pickle.dump(idx2word,f,pickle.HIGHEST_PROTOCOL)

In [69]:
keep.columns

Index(['HADM_ID', 'TEXT', 'CATEGORY', 'DIAG_CODES', 'PROC_CODES', 'ind',
       'padded_tokens', 'CLEANED TEXT', 'CLEAN_WORDS', 'STEM_WORDS',
       'CODED_TEXT'],
      dtype='object')

In [70]:
keep2=keep[['HADM_ID', 'CATEGORY', 'TEXT', 'DIAG_CODES', 'PROC_CODES', 'CODED_DIAG',
       'CODED_PROC', 'HEADERS', 'ind', 'CLEANED TEXT', 'CLEAN_WORDS',
       'CODED_TEXT']]

KeyError: "['CODED_DIAG', 'CODED_PROC', 'HEADERS'] not in index"

In [66]:
keep2.shape

(38, 11)

In [64]:
(keep2['TEXT'])

585      Admission Date:\nDischarge Date:\nMEDICAL COND...
4408     Admission Date:\nDischarge Date:\nMEDICAL COND...
4662     Admission Date: Discharge Date:    Date of Bir...
5067     Admission Date:\nDischarge Date:\nFINAL REPORT...
6234     Admission Date:\nDischarge Date:\nMEDICAL COND...
6486     Admission Date:\nDischarge Date:\nMEDICAL COND...
6521     PATIENT/TEST INFORMATION:\n-----\n<    4.3  27...
7382     Admission Date:\nDischarge Date:\nPROTEUS SPEC...
7746     Admission Date:\nDischarge Date:\nMEDICAL COND...
8654     Admission Date:\nDischarge Date:\n-----\nProte...
8692     Admission Date:\nDischarge Date:\n-----\nProte...
8742     Admission Date:\nDischarge Date:\nMEDICAL COND...
8932     be sinus tachycardia, but consider also ectopi...
9651     Admission Date:\nDischarge Date:\nMEDICAL COND...
9757     Admission Date:\nDischarge Date:\nMEDICAL COND...
9774     PATIENT/TEST INFORMATION:\nMEDICAL CONDITION: ...
10615    Admission Date:\nDischarge Date:\n-----\nProte.